In [1]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
#from psycopg2 import extras as ex
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from sklearn.preprocessing import *
import datetime

## prepare data

In [2]:
# connect to the postgresql
db_connection = psycopg2.connect(host='127.0.0.1',dbname="chicago_business_intelligence", user="postgres" , password="12345")
cursor = db_connection.cursor()

In [3]:
# get data
select_data = (
    """
        SELECT * FROM "neighborhood_community_zip";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_1 = pd.DataFrame(result_1)
df_1.columns = ["id","geo_id","zipcode","community","neighborhood"]
df_1 = df_1[["zipcode","community","neighborhood"]]
df_1.zipcode = df_1.zipcode.astype("int")
df_1_pick = df_1.copy()
df_1_pick.columns = ["pickup_zip_code","community","neighborhood"]
df_1_drop = df_1.copy()
df_1_drop.columns = ["dropoff_zip_code","community","neighborhood"]
df_1

,zipcode,community,neighborhood
0,60601,Loop,Printer's Row
1,60602,Loop,Printer's Row
2,60603,Loop,Printer's Row
3,60604,Loop,Printer's Row
4,60605,Loop,Printer's Row
...,...,...,...
306,60652,Ashburn,Parkview
307,60652,Ashburn,Scottsdale
308,60652,Ashburn,Wrightwood
309,60656,O'Hare,Schorsch Forest View


In [4]:
# get data
select_data = (
    """
        SELECT * FROM "taxi_trip_total";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_2 = pd.DataFrame(result_1)
df_2.columns = ["id","trip_id","trip_start_timestamp","trip_end_timestamp","pickup_centroid_latitude","pickup_centroid_longitude",
                   "dropoff_centroid_latitude","dropoff_centroid_longitude","pickup_zip_code","dropoff_zip_code"]
df_2 = df_2[["trip_id","trip_start_timestamp","trip_end_timestamp","pickup_zip_code","dropoff_zip_code"]]
df_2.pickup_zip_code = df_2.pickup_zip_code.astype("int")
df_2.dropoff_zip_code = df_2.dropoff_zip_code.astype("int")
df_2

,trip_id,trip_start_timestamp,trip_end_timestamp,pickup_zip_code,dropoff_zip_code
0,d685d4584ba0ee84f76cd82d11bc621c2fe15172,2022-03-01 00:00:00+08:00,2022-03-01 00:00:00+08:00,60666,60666
1,d5e65b4514cd47fa97de827832c4942ca50064af,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60610,60657
2,b75d0e728ef64abdc898759411c3598e5a0ba446,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60657,60616
3,9f6cd473daf9f0d0c4f300d28a80306d121fc0fb,2022-03-01 00:00:00+08:00,2022-03-01 01:00:00+08:00,60610,60610
4,95e8a5e899d3bae67509c12ddc2e485f0a80b5be,2022-03-01 00:00:00+08:00,2022-03-01 00:15:00+08:00,60666,60604
...,...,...,...,...,...
3239,ce41af120ab714bb90b038f1b80c0ec51d52a76a,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60607,60603
3240,ce1e9bae00df7404637ecd854d2e8b398a8030fc,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60643,60616
3241,ce077a5c833c8381781e7ea855705ba9df9978f3,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60622,60610
3242,cdf4b63ece86bb51a653f42726bc8eb61b3b98bb,2022-02-28 22:45:00+08:00,2022-02-28 23:00:00+08:00,60610,60604


In [5]:
# get data
select_data = (
    """
        SELECT * FROM "covid_weekly_case";
    """)
# create the table
cursor.execute(select_data)
result_1 = cursor.fetchall();
# set dataframe
df_3 = pd.DataFrame(result_1)
df_3.columns = ["id","dropoff_zip_code","week_number","week_start","week_end","cases_weekly","cases_cumulative"]
df_3 = df_3[["dropoff_zip_code","week_start","cases_weekly"]]
df_3["week_start"] = df_3["week_start"].apply(lambda x:x.strftime("%Y-%m-%d"))
df_3 = df_3[df_3["dropoff_zip_code"]!="Unknown"]
df_3.dropoff_zip_code = df_3.dropoff_zip_code.astype("int")
df_3

,dropoff_zip_code,week_start,cases_weekly
0,60656,2022-03-20,20.0
1,60652,2022-03-20,13.0
2,60632,2022-03-20,10.0
3,60620,2022-03-20,16.0
4,60616,2022-03-20,28.0
...,...,...,...
495,60621,2022-01-23,50.0
496,60615,2022-01-23,70.0
497,60607,2022-01-23,93.0
498,60602,2022-01-23,4.0


## process data

In [6]:
df_2_total = df_2.merge(df_1_pick,how='inner', on='pickup_zip_code')
df_2_total["ymd"] = df_2_total["trip_start_timestamp"].apply(lambda x:x.strftime("%Y-%m-%d"))
df_2_total = df_2_total[df_2_total["community"] == "O'Hare"]
df_2_total = df_2_total[["trip_id","ymd","dropoff_zip_code"]]
df_2_total = df_2_total.merge(df_1_drop,how='inner', on='dropoff_zip_code')
df_2_total = df_2_total[["trip_id","ymd","dropoff_zip_code","community"]]
df_2_total = df_2_total.merge(df_3,how='inner', on='dropoff_zip_code')
df_2_total = df_2_total[["community","trip_id","ymd","dropoff_zip_code","week_start","cases_weekly"]]
df_2_total

,community,trip_id,ymd,dropoff_zip_code,week_start,cases_weekly
0,O'Hare,d685d4584ba0ee84f76cd82d11bc621c2fe15172,2022-03-01,60666,2022-03-20,0.0
1,O'Hare,d685d4584ba0ee84f76cd82d11bc621c2fe15172,2022-03-01,60666,2022-03-13,2.0
2,O'Hare,d685d4584ba0ee84f76cd82d11bc621c2fe15172,2022-03-01,60666,2022-03-06,1.0
3,O'Hare,d685d4584ba0ee84f76cd82d11bc621c2fe15172,2022-03-01,60666,2022-02-27,0.0
4,O'Hare,d685d4584ba0ee84f76cd82d11bc621c2fe15172,2022-03-01,60666,2022-02-20,0.0
...,...,...,...,...,...,...
31115,Morgan Park,f094f0abbcd0791adc479a4e2895d6c4c868cbd2,2022-02-28,60643,2022-02-27,12.0
31116,Morgan Park,f094f0abbcd0791adc479a4e2895d6c4c868cbd2,2022-02-28,60643,2022-02-20,21.0
31117,Morgan Park,f094f0abbcd0791adc479a4e2895d6c4c868cbd2,2022-02-28,60643,2022-02-13,25.0
31118,Morgan Park,f094f0abbcd0791adc479a4e2895d6c4c868cbd2,2022-02-28,60643,2022-02-06,26.0


## report

In [7]:
# neighborhoodsccvi_category
new_w = df_2_total.groupby(by=["dropoff_zip_code"],dropna=False)["week_start"].max().reset_index()
new_w = new_w.merge(df_2_total,how='left', on=['week_start',"dropoff_zip_code"])
new_w

,dropoff_zip_code,week_start,community,trip_id,ymd,cases_weekly
0,60601,2022-03-20,Loop,1217a033e5386f0b655f1fa7cde5644bedbab5e1,2022-02-28,20.0
1,60601,2022-03-20,Loop,b2b64728accd307bf78372479d9cdb89e002068f,2022-02-28,20.0
2,60601,2022-03-20,Loop,7b24367db7ed8d7a78e9afa8a69b96a00713fd6b,2022-02-28,20.0
3,60601,2022-03-20,Loop,66a19019f7721c304d3b1c9d8209929e4b347beb,2022-02-28,20.0
4,60601,2022-03-20,Loop,ba2836ed552e940b04f30536fb1b3ec247824925,2022-02-28,20.0
...,...,...,...,...,...,...
3759,60666,2022-03-20,O'Hare,be2615b04bc74c0637a5e89c08c4abdbc8fe4d6c,2022-02-28,0.0
3760,60666,2022-03-20,O'Hare,9a36d2f9660eb3f74ece57934c431a815ecadc8e,2022-02-28,0.0
3761,60666,2022-03-20,O'Hare,c9ffb5dabae751639a49605df7d6cfd10a6aa1d0,2022-02-28,0.0
3762,60666,2022-03-20,O'Hare,3820d7e1beb37b7270e098a828eb9ada32f75093,2022-02-28,0.0
